Daniel F Carlson Helmholtz-Zentrum Hereon 
daniel.carlson@hereon.de

This notebook demonstrates how to:
1. Generate a collection of Sentinel-2 images based on the criteria: bounding box, date range, and cloud cover
2. Compute the footprint of the image to remove partial tiles
3. Identify and remove duplicate tile IDs
4. Write tile ids to a file

This notebook uses Harmonized S2 L1C: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED

In [1]:
# Import necessary libraries
import os
import ee
import csv
import geemap
import geemap.colormaps as cm
import numpy as np
from IPython.display import display, HTML
import ipywidgets as widgets

In [2]:
# Authenticate
ee.Authenticate()
ee.Initialize()

In [3]:
#cm.list_colormaps()

In [4]:
# Define Functions
# Define a function to compute the area of image footprints
def computeFootprintArea(image):
    footprint = image.geometry()
    area_m2 = footprint.area()
    return image.set({'footprint_area_m2': area_m2})

# Function to check for duplicate images
def check_s2_duplicates(imlist,vch=False):
    im2pop = []
    for x in range(0,len(imlist)):
        chk_str = imlist[x]
        chk_str_spl = chk_str.split('_')
        chk_shrt = f"{chk_str_spl[0][0:8]}_{chk_str_spl[2]}"
        for y in range(0,len(imlist)):
            if y > x:
                sec_str = imlist[y]
                sec_str_spl = sec_str.split('_')
                sec_shrt = f"{sec_str_spl[0][0:8]}_{sec_str_spl[2]}"

                if chk_shrt == sec_shrt:
                    if vch:
                        print(f"Found duplicate: {imlist[x]} and {imlist[y]}")
                    im2pop.append(y)
    if im2pop==0:
        return None
    else:
        im2pop_un = sorted(list(tuple(im2pop)))
        for index in sorted(im2pop_un,reverse=True):
            del imlist[index]
        return imlist

In [ ]:
# Set base directory
base_dir = '/Users/dfc/Documents/MyPythonRepos/DFC_new/LABSEA_MCRO_GEE_S2'
out_dir = os.path.join(base_dir,'tile_id_out')

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Define search years and months
yrs = [2019,2020,2021,2022,2023]
mns = [[3,31], [4,30], [5,31], [6,30], [7,31], [8,31], [9,30], [10,31] ]

# Just looking (False). Or want to save (True)
make_list = True

# Set search year and month
#srch_yr = 2019
#srch_mn = '05'
# Set min/max area for S2 tiles
# Define the minimum and maximum allowable footprint area (in square meters)
min_footprint_area = 5000000000  # Minimum area 
max_footprint_area = 50000000000  # Maximum area 

# roi 
fnm = 'NAtl_ROI.csv'
fpath = os.path.join(base_dir,fnm)
csv_data = []
with open(fpath, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if not row[0][0] == 'L':
            lat, lon = float(row[1]), float(row[0])
            csv_data.append((lon, lat))
if len(csv_data) > 0:
    # Create a polygon geometry from the CSV data
    roi = ee.Geometry.Polygon(csv_data)
else:
    roi = -10
if not roi == -10:
    
    for y in yrs:
        for m in mns:
            # find S2 images in the ROI
            # Define the time range for the image search
            drng = [f"{y}-{m[0]:02}-01",f"{y}-{m[0]:02}-{m[1]}"]
            start_date = drng[0]
            end_date = drng[1]
            # Define the maximum allowable cloud cover percentage
            max_cloud_cover = 40  # Adjust as needed
            
            # initialize output file

            if make_list:
                imlist = []
                outfnm = f'{y}_{m[0]:02}_Area_Thresh_S2_TOA_image_ids.csv'
                outpath = os.path.join(out_dir,outfnm)

            
            print(f"Now looking for S2 images between {start_date} and {end_date} with up to {max_cloud_cover}% cloud cover.")

            # Create a Sentinel-2 image collection
            sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterBounds(roi) \
                .filterDate(ee.Date(start_date), ee.Date(end_date)) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))
            # Print the number of images in the collection
            print("Number of Sentinel-2 images in ROI:", sentinel2_collection.size().getInfo())
            # Map the function over the image collection to compute footprint areas
            sentinel2_collection_with_area = sentinel2_collection.map(computeFootprintArea)

            # Filter the collection based on footprint area
            area_filtered_collection = sentinel2_collection_with_area \
                .filter(ee.Filter.gt('footprint_area_m2', min_footprint_area)) \
                .filter(ee.Filter.lt('footprint_area_m2', max_footprint_area))

            # Get the size of the filtered collection (number of images)
            area_collection_size = area_filtered_collection.size().getInfo()
            print("Number of images in the filtered collection:", area_collection_size)
            if make_list:
                f = open(outpath,'w')
                f.close()
                # Convert the image collection to a client-side list
                image_list = area_filtered_collection.toList(sentinel2_collection.size())
                for i in range(image_list.size().getInfo()):
                    image_id = ee.Image(image_list.get(i)).id().getInfo()
                    imlist.append(image_id)

                imlist_out = check_s2_duplicates(imlist)
                if imlist_out is None:
                    print("Well that's awkward. Something went wrong somewhere...")
                else:
                    for il in imlist_out:
                        with open(outpath, 'a') as f:
                            print(f"Now writing {il} to file")
                            pstr = f"{il},\n"
                            f.write(pstr)
                            f.close()

Now looking for S2_SR images between 2019-03-01 and 2019-03-31 with up to 40% cloud cover.
Number of Sentinel-2 images in ROI: 610
Number of images in the filtered collection: 404
